In [1]:
from flytekit.configuration import set_flyte_config_file, platform
set_flyte_config_file("../../notebook.config")

print("Connected to {}".format(platform.URL.get()))

def print_console_url(exc):
    print("http://{}/console/projects/{}/domains/{}/executions/{}".format(platform.URL.get(), exc.id.project, exc.id.domain, exc.id.name))

Connected to localhost:30081


# Demo: Workflow without flytekit dependency in containers

This is a simple workflow where we will show how one can use flytekit to create a workflow that needs not container building, but uses an existing set of container to run logic. It also demonstrates that even in this case memoization, cataloging and input/output types are preserved.

## Step I: Lets declare a couple tasks.

### Task 1: square
Given an integer returns the squre of the integer

In [2]:
from flytekit.common.tasks.raw_container import SdkRawContainerTask
from flytekit.sdk.types import Types

square = SdkRawContainerTask(
    input_data_dir="/var/inputs",
    output_data_dir="/var/outputs",
    inputs={"val": Types.Integer},
    outputs={"out": Types.Integer},
    image="alpine",
    command=["sh", "-c", "echo $(( {{.Inputs.val}} * {{.Inputs.val}} )) | tee /var/outputs/out"],
)


### Task 2: sum
Given two integer x & y returns the sum (x + y)

In [3]:
sum = SdkRawContainerTask(
    input_data_dir="/var/flyte/inputs",
    output_data_dir="/var/flyte/outputs",
    inputs={"x": Types.Integer, "y": Types.Integer},
    outputs={"out": Types.Integer},
    image="alpine",
    command=["sh", "-c", "echo $(( {{.Inputs.x}} + {{.Inputs.y}} )) | tee /var/flyte/outputs/out"],
)


## Step II: Declare a workflow
#### Sum of Squares
Given two integers x & y, return (x^2 + y^2)
This is composed using two functions
 sum_of_squares = sum(square(x),square(y))

In [4]:
from flytekit.sdk.workflow import workflow_class, Input, Output

@workflow_class
class SumOfSquares(object):
    val1 = Input(Types.Integer)
    val2 = Input(Types.Integer)
    sq1 = square(val=val1)
    sq2 = square(val=val2)
    sm = sum(x=sq1.outputs.out, y=sq2.outputs.out)
    sum_of_squares = Output(sm.outputs.out, sdk_type=Types.Integer)
    
SumOfSquares_lp = SumOfSquares.create_launch_plan() 

### Step III: Register the workflow & launch plan
Register the workflow under project: flyteexamples 

In [5]:
# Currently in flytekit you have to register the tasks first and then register the workflow
square.register(name="square", project="flyteexamples", domain="development", version="x")
sum.register(name="sum", project="flyteexamples", domain="development", version="x")

# It will use the name of the tasks to find the registrations
SumOfSquares.register(name="SumOfSquares", project="flyteexamples", domain="development", version="x")
SumOfSquares_lp.register(name="SumOfSquares", project="flyteexamples", domain="development", version="x")

'lp:flyteexamples:development:SumOfSquares:x'

In [6]:
exc = SumOfSquares_lp.execute("flyteexamples", "development", inputs={"val1": 3, "val2": 2})
print_console_url(exc)

/Users/kumare/.virtualenvs/flytekit/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method execute. (Use launch instead) -- Deprecated since version 0.9.0.
  """Entry point for launching an IPython kernel.


http://localhost:30081/console/projects/flyteexamples/domains/development/executions/f8f61c1361dc246bca1c


In [7]:
exc.wait_for_completion()

In [8]:
exc.outputs

{'sum_of_squares': 13}